# **Let's learn about decorators** 

In [1]:
import time
from functools import partial, wraps, lru_cache

A decorator is a callable that takes another function as argument.The decorator may perform some processing with the decorated function, and
returns it or replaces it with another function or callable object.

```python
@decorate_func
def target():
    print('running target()')
    
#############################
    
def target():
    print('running target()')
    
target = decorate_func(target)
```


- Strictly speaking, decorators are just syntactic sugar. 

- Usung decorators is  convenient, especially when doing metaprogramming—changing program behavior at runtime.

- They are executed immediately when a module is loaded.

## **Closures** 

In [6]:
def historical_print():
    
    hist_lst = []
    
    def inner_print(string):
        hist_lst.append(string)
        print(" ".join(hist_lst))
        
    print(f"style_print closure:\n{inner_print.__closure__}")
    print()
    print(f"super_print closure:\n{historical_print.__closure__}")

    return inner_print

In [8]:
printer = historical_print()

style_print closure:
(<cell at 0x7f58583aeac0: list object at 0x7f58583b3b80>,)

super_print closure:
None


- The ```inner_print``` function finds the ```hist_lst``` in their closure.

In [9]:
printer("I")

I


In [10]:
printer("like")

I like


In [11]:
printer("it")

I like it


## **Free variables** 

In [12]:
lst = [1,2,3,4,5,6]
soma = sum(lst)
size = len(lst)
print(soma/size)

3.5


In [13]:
def make_averager_v1():
    series = []
    def averager_v1(new_value):
        series.append(new_value)
        total = sum(series)
        print(total/len(series))
    return averager_v1

- The closure for ```averager``` extends the scope of that function.
- ```series``` is a **local variable** of ```make_averager```
- ```series``` is a **free variable** of ```averager```. This means a variable is not bound in the local scope.

In [14]:
avg = make_averager_v1()

In [15]:
avg(10)
avg(20)
avg(30)

10.0
15.0
20.0


In [16]:
print(f"Local variables: {avg.__code__.co_varnames}")
print(f"Free variables: {avg.__code__.co_freevars}")
print(f"Closure: {avg.__closure__[0].cell_contents}")

Local variables: ('new_value', 'total')
Free variables: ('series',)
Closure: [10, 20, 30]


- Each item in the closure corresponds to a name in the free variables.

In [17]:
def make_averager_v2():
    count = 0
    total = 0
    def averager_v2(new_value):
        count += 1
        total += new_value
        print(total / count)
    return averager_v2

In [18]:
avg = make_averager_v2()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

The ```+=``` operator means  ```count = count + 1```, but count is a number or any immutable type, which is not allowed. Immutable types like numbers, strings, tuples, etc., can only be read.

In [19]:
def make_averager_v3():
    count = [0]
    total = [0]
    def averager_v3(new_value):
        count[0] += 1
        total[0] += new_value
        print(total[0] / count[0])
    return averager_v3

In [20]:
avg = make_averager_v3()
avg(10)
avg(20)
avg(30)

10.0
15.0
20.0


In [21]:
def make_averager_v4():
    count = 0
    total = 0
    def averager_v4(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        print(total / count)
    return averager_v4

In [22]:
avg = make_averager_v4()
avg(10)
avg(20)
avg(30)

10.0
15.0
20.0


In [23]:
print(f"Local variables: {avg.__code__.co_varnames}")
print(f"Free variables: {avg.__code__.co_freevars}")
print(f"Closure: {[i.cell_contents for i in avg.__closure__]}")

Local variables: ('new_value',)
Free variables: ('count', 'total')
Closure: [3, 60]


## **Decorators** 

### **Decorators are executed in import time**

In [24]:
print("""Even tough, the functions f1 and f2 are not called the registry already have both function addrees\n""")

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
print(f"\nRegistry: {registry}")

Even tough, the functions f1 and f2 are not called the registry already have both function addrees

running register(<function f1 at 0x7f58582cd0d0>)
running register(<function f2 at 0x7f58582cd670>)

Registry: [<function f1 at 0x7f58582cd0d0>, <function f2 at 0x7f58582cd670>]


### **Example 1**

In [ ]:
import time

def clock(func):
    
    def clocked(*args): #
        t0 = time.perf_counter()
        result = func(*args) #
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    print(f"Free variables: {clocked.__code__.co_freevars}")
    print(f"Closure: {clocked.__closure__[0].cell_contents}")
    return clocked #
    
@clock
#Recursao caudal 
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

The same as:
```python    
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial = clock(factorial)
``` 

In [ ]:
_=factorial(4)

In [ ]:
@clock
#Recursao 
def fibonnaci(n):
    return 1 if n < 2 else fibonnaci(n-1) + fibonnaci(n-2)
_ = fibonnaci(4)

In [ ]:
@clock
@lru_cache()
#Memoization
def fibonnaci(n):
    return 1 if n < 2 else fibonnaci(n-1) + fibonnaci(n-2)
print(fibonnaci(10))

In [ ]:
print(f"Factorial: {factorial.__name__}")
print(f"Fibonnaci: {fibonnaci.__name__}")

This is the typical behavior of a decorator: **it replaces the decorated function with a new function that accepts the same arguments and (usually) returns whatever the decorated function was supposed to return**.

### **Example 2**

In [ ]:
def best_clock(func):
    @wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

Wrappers:

```python
@functools.wraps(func)
def g():
    pass
```

- it copies the ```__module__```, ```__name__```, ```__qualname__```,  ```__doc__```, and  ```__annotations__``` attributes of ```func``` on ```g```. This default list is in WRAPPER_ASSIGNMENTS, you can see it in the functools source.

- it updates the ```__dict__``` of g with all elements from ```func.__dict__```. (see WRAPPER_UPDATES in the source)

- it sets a new ```__wrapped__ = func``` attribute on ```g```



Ref: https://stackoverflow.com/questions/308999/what-does-functools-wraps-do

In [ ]:
@best_clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)
_=factorial(5)

In [ ]:
@lru_cache()
@best_clock
#Recursao 
def fibonnaci(n):
    return 1 if n < 2 else fibonnaci(n-1) + fibonnaci(n-2)
_=fibonnaci(5)

In [ ]:
print(f"Factorial: {factorial.__name__}")
print(f"Fibonnaci: {fibonnaci.__name__}")

## **Decorators with parameters**

### **Example 1** 

In [ ]:
def argument_clock(is_print=True):
    def decorate(func):
        @wraps(func)
        def clocked(*args, **kwargs):
            result = func(*args, **kwargs)
            if is_print:
                t0 = time.time()
                elapsed = time.time() - t0
                name = func.__name__
                arg_lst = []
                if args:
                    arg_lst.append(', '.join(repr(arg) for arg in args))
                if kwargs:
                    pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
                    arg_lst.append(', '.join(pairs))
                arg_str = ', '.join(arg_lst)
                print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
                return result
            else:
                return result
        return clocked    
    return decorate


In [ ]:
@lru_cache()
@argument_clock(is_print=False)
def fibonnaci(n):
    return 1 if n < 2 else fibonnaci(n-1) + fibonnaci(n-2)
fibonnaci(4)

In [ ]:
@lru_cache()
@argument_clock(is_print=True)
def fibonnaci(n):
    return 1 if n < 2 else fibonnaci(n-1) + fibonnaci(n-2)
_=fibonnaci(4)

### **Example 2 - Valentin Berlier**

In [ ]:
def print_result(func=None, prefix=''):
    if func is None:
        return partial(print_result, prefix=prefix)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{prefix}{result}')
        return result
    return wrapper

In [ ]:
@print_result
def add(a, b):
    return a + b

add(2, 3)  # outputs '5'

@print_result()
def add(a, b):
    return a + b

add(2, 3)  # outputs '5'

@print_result(prefix='The return value is ')
def add(a, b):
    return a + b

add(2, 3)  # outputs 'The return value is 5'

## **References** 

Book:
- Fluent Python - by Luciano Ramalho